In [1]:
import import_ipynb
from matscholar import MatScholarProcess

def process_file(text):
    zt_answer = {}
    for i, z in enumerate(text):
        z_split = z.split(', ')
        if z_split[1] != 'NaN':
            mat_name = z_split[0][1:-1]
            mat_rec = float(z_split[1])
            if mat_name in zt_answer.keys():
                if mat_rec > zt_answer[mat_name]:
                    zt_answer[mat_name] = mat_rec
            else:
                zt_answer[mat_name] = mat_rec

    sorted_zt = sorted(zt_answer.items(), key=lambda item: item[1], reverse=True)
    zt_rank = {}
    rank = 1
    processor = MatScholarProcess()
    for s in sorted_zt:
        processed, _ = processor.process(s[0])
        if processed[0] not in zt_rank.keys():
            zt_rank[processed[0]] = {}
            zt_rank[processed[0]]['rank'] = rank
            zt_rank[processed[0]]['value'] = s[1]
            rank += 1
    return zt_rank

with open('PF.txt', 'r', encoding='utf-8') as f:
    pf = str(f.readlines())
pf_split = pf[4:-4].split('], [')
pf_rank = process_file(pf_split)
print(len(pf_rank))
# 281 in max power factor: # 1 Bi2Te3 0.006728 # 2 Co2NaO4 0.0066667 # 3 HfNi4Sn4Ti2Zr 0.0065971

281


In [1]:
import json
with open('smaller_3.json', 'r', encoding='utf-8') as f:
    smaller = json.load(f)
print(len(smaller))

77593


In [5]:
# output_sims
# embbeding_sims
with open('output_sims.json', 'r', encoding='utf-8') as f:
    sims = json.load(f)

sim_dict = {}
rank = 1
for s in sims:
    if s[0] in pf_rank.keys() and s[0] not in smaller:
        sim_dict[s[0]] = {}
        sim_dict[s[0]]['rank'] = rank
        sim_dict[s[0]]['value'] = s[1]
        rank += 1
print(len(sim_dict.keys()))

83


In [6]:
sim_in_pf = {}
r = 1
for p in pf_rank.keys():
    if p in sim_dict.keys() and p not in smaller:
        sim_in_pf[p] = r
        r += 1
print(len(sim_in_pf.keys()))

83


In [7]:
ori = []
answer = []
for i in sim_in_pf.keys():
    ori.append(sim_dict[i]['rank'])
    answer.append(sim_in_pf[i])
print(len(ori))

83


In [8]:
from scipy import stats
stats.spearmanr(ori, answer)

SpearmanrResult(correlation=0.6576130305192895, pvalue=1.4413891254169311e-11)

In [2]:
with open('fre_dis.json', 'r', encoding='utf-8') as f:
    fre_dis = json.load(f)

In [4]:
print(fre_dis['Bi2Te3'])

{'p_t': 759, 'p_10': 0.2608695652173913, 'p11_13': 0.16073781291172595, 'p14_16': 0.14756258234519104, 'p17_19': 0.22793148880105402, 'p20_21': 0.2015810276679842, 'd': 0.9077733860342556, 'd_10': 0.9393939393939394, 'd11_13': 0.9016393442622951, 'd14_16': 0.875, 'd17_19': 0.9017341040462428, 'd20_21': 0.9019607843137255}


In [10]:
for i in sim_in_pf.keys():
    if i in fre_dis.keys() and i not in smaller:
        if fre_dis[i]['p_t'] > 3 and fre_dis[i]['p20_21'] == 0 and fre_dis[i]['d'] <= 0.7:
            print(fre_dis[i].keys())
            print(i, sim_dict[i]['rank'], sim_in_pf[i], fre_dis[i]['p20_21'], fre_dis[i]['d'])

dict_keys(['p_t', 'p_10', 'p11_13', 'p14_16', 'p17_19', 'p20_21', 'd', 'd_10', 'd11_13', 'd14_16', 'd17_19', 'd20_21'])
CuNd2O4 71 70 0.0 0.5


In [ ]:
# 如果p_t < 4, freq 20_21 + freq 17_19 = 1，d > 0.9, 往前放-10 
Bi9Sb 22 8 1.0 1.0

# 如果p_t < 4, d < 1，freq 20_21 and freq 17_19 = 0 and d < 0.7 往后放+10 
Nd2O7Ru2 69 76 0.0 0.5

# 如果p_t > 3, freq 20_21 >= 0.25
# 如果d > 0.9, 往前放20
GeSi4 58 24 0.2857142857142857 1.0
# 如果0.7 < d <= 0.9, 往前放10
Cu2Se 36 35 0.4198019801980198 0.8653465346534653
CaMnO3 44 47 0.25139664804469275 0.8491620111731844
Bi22Sb3 30 5 0.2857142857142857 0.7142857142857143

# 如果p_t > 3, freq 20_21 = 0
# 如果d > 0.9, 往后放10
Bi8K2Se13 14 33 0.0 1.0
BaO3Pb 64 56 0.0 1.0
CuO2Rh 37 61 0.0 1.0
Al2Ca5Sb6 23 66 0.0 0.9772727272727273
# 其余往后放20
CeCoFe3Sb12 3 19 0.0 0.75
CoFe3LaSb12 17 27 0.0 0.8888888888888888
CeFe4Sb12 35 28 0.0 0.8571428571428571
Ce2CoFe7Sb24 18 41 0.0 0.75
Ca9Mn10O30Yb 20 53 0.0 0.8260869565217391
CuLa2O4 60 65 0.0 0.875
CuNd2O4 71 70 0.0 0.5

In [11]:
update_rank = {}
for i in sim_in_pf.keys():
    update_rank[i] = sim_dict[i]['rank']
    if i in fre_dis.keys() and i not in smaller:
        if fre_dis[i]['p_t'] < 4 and (fre_dis[i]['p20_21']+fre_dis[i]['p17_19'] == 1) and fre_dis[i]['d'] > 0.9:
            update_rank[i] = sim_dict[i]['rank']-10
        if fre_dis[i]['p_t'] < 4 and fre_dis[i]['p20_21'] == 0 and fre_dis[i]['p17_19']==0 and fre_dis[i]['d'] < 0.7:
            update_rank[i] = sim_dict[i]['rank']+10
        if fre_dis[i]['p_t'] > 3 and fre_dis[i]['p20_21'] > 0.25:
            if fre_dis[i]['d'] > 0.9:
                update_rank[i] = sim_dict[i]['rank']-20
            elif fre_dis[i]['d'] > 0.7:
                update_rank[i] = sim_dict[i]['rank']-10
        if fre_dis[i]['p_t'] > 3 and fre_dis[i]['p20_21'] == 0:
            if fre_dis[i]['d'] > 0.9:
                update_rank[i] = sim_dict[i]['rank']+10
            else:
                update_rank[i] = sim_dict[i]['rank']+20

In [12]:
pre = []
answer = []
for i in sim_in_pf.keys():
    pre.append(update_rank[i])
    answer.append(sim_in_pf[i])
print(len(pre))

83


In [13]:
stats.spearmanr(pre, answer)

SpearmanrResult(correlation=0.6951278415704903, pvalue=3.0784399412926636e-13)

In [19]:
material_83 = []
our_list = sorted(update_rank.items(), key=lambda item:item[1])
sim_list = sorted(sim_dict.items(), key=lambda item:item[1]['rank'])
answer_list = sorted(sim_in_pf.items(), key=lambda item:item[1])
for i, ent in enumerate(our_list):
    material_83.append(ent[0])
    print(i+1, ent[0], sim_list[i][0], answer_list[i][0])

1 Bi2Te3 Bi2Te3 Bi2Te3
2 Bi23Sb2 Ca50Mn49NbO150 Co2NaO4
3 Ca50Mn49NbO150 CeCoFe3Sb12 HfNi4Sn4Ti2Zr
4 Hf7Ni20Sn20Ti6Zr7 Hf7Ni20Sn20Ti6Zr7 Hf7Ni20Sn20Ti6Zr7
5 HfNi4Sn4Ti2Zr HfNi4Sn4Ti2Zr Bi22Sb3
6 Co20InSb60 Co20InSb60 Bi23Sb2
7 AlCa3Sb3 AlCa3Sb3 Co20InSb60
8 NiSnTi NiSnTi Bi9Sb
9 AlO50Zn49 AlO50Zn49 Sb2Te3
10 Co20La19O60Sr Co20La19O60Sr Ca2Co2O5
11 Bi9Sb Bi23Sb2 Ni50SbSn49Zr50
12 Sb2Te3 Sb2Te3 NiSnZr
13 Ca3Co4O9 Ca3Co4O9 Ba39Nb200O600Sr61
14 Co40InSb120 Bi8K2Se13 Ca3Co4O9
15 HfNi2Sn2Zr Co40InSb120 Mg2Si
16 Bi4CsTe6 HfNi2Sn2Zr HfNi2Sn2Zr
17 O1361W500 CoFe3LaSb12 NiSnTi
18 Bi22Sb3 Ce2CoFe7Sb24 Ba4Ga8Ge15
19 NiSnZr O1361W500 CeCoFe3Sb12
20 CeCoFe3Sb12 Ca9Mn10O30Yb BiCoO4Sr
21 Bi8K2Se13 NiSnZr AlO20Zn19
22 MnSb11Yb14 Bi9Sb Co40InSb120
23 Co2NaO4 Al2Ca5Sb6 Bi83Sb17
24 Sb3Zn4 MnSb11Yb14 GeSi4
25 Cu2Se Co2NaO4 AlO50Zn49
26 Ba4Ga8Ge15 Sb3Zn4 GeSi9
27 Ba4Ga8Sn15 Ba4Ga8Ge15 CoFe3LaSb12
28 Mo3NiS4 Bi4CsTe6 CeFe4Sb12
29 Ni50SbSn49Zr50 Ba4Ga8Sn15 Bi4CsTe6
30 Ba4Ga8Si15 Bi22Sb3 Ga8Ge15Sr4
31 Al2Ca5Sb

In [20]:
print(material_83)

['Bi2Te3', 'Bi23Sb2', 'Ca50Mn49NbO150', 'Hf7Ni20Sn20Ti6Zr7', 'HfNi4Sn4Ti2Zr', 'Co20InSb60', 'AlCa3Sb3', 'NiSnTi', 'AlO50Zn49', 'Co20La19O60Sr', 'Bi9Sb', 'Sb2Te3', 'Ca3Co4O9', 'Co40InSb120', 'HfNi2Sn2Zr', 'Bi4CsTe6', 'O1361W500', 'Bi22Sb3', 'NiSnZr', 'CeCoFe3Sb12', 'Bi8K2Se13', 'MnSb11Yb14', 'Co2NaO4', 'Sb3Zn4', 'Cu2Se', 'Ba4Ga8Ge15', 'Ba4Ga8Sn15', 'Mo3NiS4', 'Ni50SbSn49Zr50', 'Ba4Ga8Si15', 'Al2Ca5Sb6', 'BiTe6Tl9', 'CaMnO3', 'CoFe3LaSb12', 'GeSi4', 'Ce2CoFe7Sb24', 'LaO60Sr19Ti20', 'Cu99Se50', 'Ca9Mn10O30Yb', 'Mo3Te4', 'NbO15Sr5Ti4', 'Mg2Si', 'LaO30Sr9Ti10', 'Ga8Ge15Sr4', 'Bi83Sb17', 'Ca2Co2O5', 'CuO2Rh', 'SnTe5Tl2', 'AgCrSe2', 'BiCoO4Sr', 'LaO15Sr4Ti5', 'Ca9CeMn10O30', 'O29W10', 'CoLaO3', 'GeSi9', 'CeFe4Sb12', 'NiO50Zn49', 'Cr5Se8Tl', 'Cu10La19O40Sr', 'In2O3', 'CaCuMn6O12', 'Cu2Mo3S4', 'Bi2O7Ru2', 'Cr19Cu20MgO40', 'Cu20La37O80Sr3', 'CrCuO2', 'AlO20Zn19', 'LiMn2O4', 'OZn', 'Ba39Nb200O600Sr61', 'BaO3Pb', 'LaNiO3', 'O3W', 'O7Ru2Y2', 'CaMn2O6Sm', 'Ca9LaMn10O30', 'Nd2O7Ru2', 'CuLa2O4', 'CaMn